# Capstone project notebook

## Problem 3

### What Is the Relationship between Housing Characteristics and Complaints?
The goal of this exercise is to find the answer to the Question 3 of the problem statement: 

### Does the Complaint Type that you identified in response to Question 1 have an obvious relationship with any particular characteristic or characteristic of the Houses?

In this exercise, use the 311 dataset.

You also need to read back the PLUTO dataset from Cloud Object Store that you saved previously in the course. Use the PLUTO dataset for the borough that you already identified to focus on the last exercise.Ensure that you use only a limited number of fields from the dataset so that you are not consuming too much memory during your analysis.

The recommended fields are Address, BldgArea, BldgDepth, BuiltFAR, CommFAR, FacilFAR, Lot, LotArea, LotDepth, NumBldgs, NumFloors, OfficeArea, ResArea, ResidFAR, RetailArea, YearBuilt, YearAlter1, ZipCode, YCoord, and XCoord.

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_cba83a820ee941cd921cc2bbfefd15eb = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='56RsTx8E6jPTOm05rXcEWp-YR-lpps36XB-mae6-wNk9',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_cba83a820ee941cd921cc2bbfefd15eb.get_object(Bucket='edx1-donotdelete-pr-ffppmpbmudcobi',Key='nyc311.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_311 = pd.read_csv(body)
df_311.head()

### Read Bronx file

To prevent loading unnecesary data, we select the columns to load

In [ ]:
body = client_cba83a820ee941cd921cc2bbfefd15eb.get_object(Bucket='edx1-donotdelete-pr-ffppmpbmudcobi',Key='bronxs.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
cols_to_read = [  'Address',
                  'BldgArea',
                  'BldgDepth',
                  'BuiltFAR',
                  'CommFAR',
                  'FacilFAR',
                  'Lot', 
                  'LotArea',
                  'LotDepth',
                  'NumBldgs',
                  'NumFloors',
                  'OfficeArea',
                  'ResArea',
                  'ResidFAR',
                  'RetailArea',
                  'YearBuilt',
                  'YearAlter1',
                  'ZipCode',
                  'YCoord',
                  'XCoord']
df_bronx_info = pd.read_csv(body, usecols=cols_to_read)

In [ ]:
df_bronx_info.head()

### We create a subset pandas dataframe

With only the values to study, namely borough, address and location. After that we make and encoding setting BRONX as 1 and all others as 0


In [ ]:
df_bronx_incidents = df_311[['complaint_type', 'incident_address', 'latitude', 'longitude', 'unique_key']].loc[df_311['borough'] == 'BRONX']
print('Number of Bronx incidents',df_bronx_incidents['unique_key'].count(),sep=' ')
#df_bronx_incidents.set_index('unique_key', inplace=True)
df_bronx_incidents.head()

In [ ]:
df_bronx_incidents['complaint_type'] = (df_bronx_incidents['complaint_type'] == 'HEAT/HOT WATER').astype(int)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df_bronx_incidents.groupby('complaint_type').agg('complaint_type').count().plot.bar()

### We need to join incidents with building information

A left inner join is what we need

In [ ]:
df_bronx = pd.merge(df_bronx_incidents, df_bronx_info, how='left', left_on=['incident_address'], right_on=['Address'])

In [ ]:
print('Number of Bronx incidents and building information',df_bronx['unique_key'].count(),sep=' ')

### There are incident streets not available in the pluto file, so we just drop them

This is a problem with cardinality.

In [ ]:
df_bronx.dropna(inplace=True)

### Let's get rid of the addresses

Let's see how many null values we have in the dataframe, and then eliminate them. Also we can select Lot as the index

In [ ]:
df_bronx.drop(['Address', 'incident_address'], axis=1, inplace=True)

### Eliminate duplicates

There are a lot of duplicates. We are cleaning them and it results in *_1,211,609_* rows

In [ ]:
df_bronx.drop_duplicates(inplace=True)

In [ ]:
print('Number of Bronx incidents with no duplicates',df_bronx['unique_key'].count(),sep=' ')

### Let's define dependant and independant variables

In [ ]:
import numpy as np

y = np.asarray(df_bronx['complaint_type'])
predictors = df_bronx.columns.difference(['complaint_type'])
X = df_bronx[predictors]

In [ ]:
X.head(10)

In [ ]:
from sklearn import preprocessing
X_norm = preprocessing.StandardScaler().fit(X).transform(X)

In [ ]:
from scipy.stats import pearsonr

### Let's calculate correlations

Pearson correlation. As we want to know the importance as predictor, the absolute value is the indicator of correlation (ignoring if it is possitive or negative

In [ ]:
correlations = []
for variable in df_bronx[df_bronx.columns.difference(['complaint_type'])]:
    corr, p_value = pearsonr(y, X[variable])
    correlations.append([variable,corr,p_value])
df_correlations = pd.DataFrame.from_records(correlations, columns=['predictor','Pearson correlation','p value'])


In [ ]:
df_correlations.sort_values('Pearson correlation').head()

In [ ]:
df_correlations.sort_values('Pearson correlation', ascending=False).head()

### Resulting in:

1. NumFloors (+)
2. BldgDepth (+)
3. ResidFAR (+)
3. FacilFAR (+)
3. XCoord (-)

In [ ]:
df_bronx.plot.scatter(x='NumFloors', y='BldgDepth', c='complaint_type')

In [ ]:
df_bronx['NumFloors'].plot.bar()